In [15]:
# Select the communes in the Petite Couronne (92,93,94,75)
# Ignore and replace Communes_PC.json with Communes_IDF.json the second cell

import numpy as np 
import pandas as pd 
import geopandas as gpd 
import branca.colormap as cm
import folium 
import datetime
import json

with open('Communes_IDF.json','r') as data_file:
    data = json.load(data_file)
    
to_delete = []
for feature in data['features']:
    if not (feature['properties']['C_INSEE']//1000) in [92,93,94,75]:
        to_delete.append(feature)
    
for feature in to_delete:
    data['features'].remove(feature)
        
with open('Communes_PC.json','w') as data_file:
    json.dump(data,data_file)

In [16]:
# Generates a gradient map based on the accessibility data

import folium
from folium.plugins import TimestampedGeoJson, TimeSliderChoropleth
import pandas as pd

#merge csvs : awk '(NR==1) || (FNR>1)' n*.csv > combined.csv PUIS awk '(NR==1) || (FNR>1)' combined*.csv > final_merged.csv

commune_geo = "Communes_IDF.json"
m = folium.Map(location=[48.8626304852,2.33629344655])

commune_data = pd.read_csv("./csv/merged.csv", sep=';')
#commune_data['Access'] = np.log10(commune_data['Access']+1)


In [17]:
region_list = commune_data['Name'].unique().tolist()
commune_data[commune_data['Name']==region_list[13]]

,Code_INSEE,Name,Coord,Access,time
13,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4842.33,03-08-2021 06:00
166,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4725.20,03-08-2021 06:10
319,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4705.00,03-08-2021 07:40
472,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4717.33,03-08-2021 07:50
625,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4812.93,03-08-2021 06:20
778,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4678.93,03-08-2021 06:30
931,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4671.00,03-08-2021 06:40
1084,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4762.73,03-08-2021 06:50
1237,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4649.93,03-08-2021 07:00
1390,94055,ORMESSON SUR MARNE,"48.7854562744,2.53924214317",4761.33,03-08-2021 07:10


In [18]:
commune_data['date_sec'] = pd.to_datetime(commune_data['time'], format='%m-%d-%Y %H:%M', errors='coerce').values.astype(int) // 10**6
for k in range(len(commune_data['date_sec'])):
    commune_data['date_sec'][k]=datetime.datetime.strptime(commune_data['time'][k], "%m-%d-%Y %H:%M").timestamp()

max_colour = max(commune_data['Access'])
min_colour = min(commune_data['Access'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
commune_data['colour'] = commune_data['Access'].map(cmap)

region_list = commune_data['Name'].unique().tolist()
region_idx = range(len(region_list))

style_dict = {}
for i in region_idx:
    region = region_list[i]
    result = commune_data[commune_data['Name'] == region]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date_sec']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

<ipython-input-18-1cdc8e863b35>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_data['date_sec'][k]=datetime.datetime.strptime(commune_data['time'][k], "%m-%d-%Y %H:%M").timestamp()


In [19]:
commune_data

,Code_INSEE,Name,Coord,Access,time,date_sec,colour
0,75119,PARIS 19,"48.886868623,2.38469432787",3291.53,03-08-2021 06:00,1615183200,#fe923eff
1,92046,MALAKOFF,"48.8169695977,2.29693599517",4207.93,03-08-2021 06:00,1615183200,#fd512aff
2,93074,VAUJOURS,"48.9324772605,2.5810025704",5266.93,03-08-2021 06:00,1615183200,#dc141eff
3,94071,SUCY EN BRIE,"48.7656456593,2.5330560803",4482.60,03-08-2021 06:00,1615183200,#f53f26ff
4,75115,PARIS 15,"48.8401554186,2.29355937244",3842.07,03-08-2021 06:00,1615183200,#fd6c32ff
...,...,...,...,...,...,...,...
1831,93049,NEUILLY PLAISANCE,"48.8643287852,2.51040249898",4264.67,03-08-2021 07:30,1615188600,#fc4d29ff
1832,94034,FRESNES,"48.7571343016,2.32612839743",4315.08,03-08-2021 07:30,1615188600,#fa4a29ff
1833,92004,ASNIERES SUR SEINE,"48.9153530123,2.2880384663",4196.00,03-08-2021 07:30,1615188600,#fd522bff
1834,93027,LA COURNEUVE,"48.9322569546,2.39978064801",3922.50,03-08-2021 07:30,1615188600,#fd6631ff


In [20]:
countries = gpd.read_file('destination_data.shp')
regions_df = countries['geometry']
regions_gdf = gpd.GeoDataFrame(regions_df)
regions_gdf = regions_gdf.drop_duplicates().reset_index()


slider_map = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron', location=[48.8626304852,2.33629344655])

_ = TimeSliderChoropleth(
    data=regions_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Accessibilité, s"
slider_map.save(outfile='./csv/TimeSliderChoropleth.html')

In [21]:
'''# Generates a gradient map based on the accessibility data

import folium
import pandas as pd


commune_geo = "Communes_PC.json"
m = folium.Map(location=[48.8626304852,2.33629344655])

commune_data = pd.read_csv("csv/final_merged.csv", sep=';')


folium.Choropleth(
    geo_data=commune_geo,
    name="choropleth",
    data=commune_data,
    columns=["Code_INSEE", "Access"],
    key_on="feature.properties.C_INSEE",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Accessibility, s",
).add_to(m)

folium.LayerControl().add_to(m)

m.save("map.html")
m


'''



'# Generates a gradient map based on the accessibility data\n\nimport folium\nimport pandas as pd\n\n\ncommune_geo = "Communes_PC.json"\nm = folium.Map(location=[48.8626304852,2.33629344655])\n\ncommune_data = pd.read_csv("csv/final_merged.csv", sep=\';\')\n\n\nfolium.Choropleth(\n    geo_data=commune_geo,\n    name="choropleth",\n    data=commune_data,\n    columns=["Code_INSEE", "Access"],\n    key_on="feature.properties.C_INSEE",\n    fill_color="YlGn",\n    fill_opacity=0.7,\n    line_opacity=0.2,\n    legend_name="Accessibility, s",\n).add_to(m)\n\nfolium.LayerControl().add_to(m)\n\nm.save("map.html")\nm\n\n\n'